# Toronto Neighborhoods

- imports

In [1]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
import folium
import json
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

- will use 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
- check for status code and headers

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
result = requests.get(url)
print(url)
print(result.status_code)
print(result.headers)

https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
200
{'Date': 'Sun, 23 Dec 2018 00:10:56 GMT', 'Content-Type': 'text/html; charset=UTF-8', 'Content-Length': '15354', 'Connection': 'keep-alive', 'Server': 'mw1320.eqiad.wmnet', 'X-Content-Type-Options': 'nosniff', 'P3P': 'CP="This is not a P3P policy! See https://en.wikipedia.org/wiki/Special:CentralAutoLogin/P3P for more info."', 'X-Powered-By': 'HHVM/3.18.6-dev', 'Content-language': 'en', 'Last-Modified': 'Fri, 07 Dec 2018 05:21:50 GMT', 'Backend-Timing': 'D=110520 t=1545369710028179', 'Vary': 'Accept-Encoding,Cookie,Authorization,X-Seven', 'Content-Encoding': 'gzip', 'X-Varnish': '120822320 116111504, 776139700 335184103', 'Via': '1.1 varnish (Varnish/5.1), 1.1 varnish (Varnish/5.1)', 'Age': '154145', 'X-Cache': 'cp1081 hit/9, cp1075 hit/18', 'X-Cache-Status': 'hit-front', 'Server-Timing': 'cache;desc="hit-front"', 'Strict-Transport-Security': 'max-age=106384710; includeSubDomains; preload', 'Set-Cookie': 'WMF-Last-Acce

- will get data + clean it

In [3]:
soup = BeautifulSoup(result.content, 'html.parser')
table = soup.find('table')
trs = table.find_all('tr')
rows = []
for tr in trs:
    i = tr.find_all('td')
    if i:
        rows.append(i)
        
lst = []
for row in rows:
    postalcode = row[0].text.rstrip()
    borough = row[1].text.rstrip()
    neighborhood = row[2].text.rstrip()
    if borough != 'Not assigned':
        if neighborhood == 'Not assigned':
            neighborhood = borough
        lst.append([postalcode, borough, neighborhood])

# lst

- convert into a dataframe

In [4]:
cols = ['PostalCode', 'Borough', 'Neighborhood']
df = pd.DataFrame(lst, columns=cols)
print(df.shape)
# df[df.duplicated(['PostalCode'], keep=False)] - this would have shown the duplicate PostalCodes

(212, 3)


- custom groupby / agg to merge Neighborhoods
    - groupby PostalCode, keep the first Borough and join() Neighborhoods

In [5]:
df = df.groupby('PostalCode').agg(
    {
        'Borough':'first', 
        'Neighborhood': ', '.join,}
    ).reset_index()

- will check if 'M5A' example is done correctly

In [6]:
df.loc[df['PostalCode'] == 'M5A']

,PostalCode,Borough,Neighborhood
53,M5A,Downtown Toronto,"Harbourfront, Regent Park"


- will check the df.shape

In [7]:
df.shape

(103, 3)

- will read geo data from a CSV
- need to rename a column so the merge (below) will work

In [8]:
dfgeo = pd.read_csv("Geospatial_Coordinates.csv")
dfgeo.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)

- need to merge 2 dataframes based on a PostalCode column

In [9]:
df2 = pd.merge(df, dfgeo, on="PostalCode", how='left')

- let's check if the example returns the expected value(s)

In [10]:
df2.loc[df2['PostalCode'] == 'M5G']

,PostalCode,Borough,Neighborhood,Latitude,Longitude
57,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383


In [11]:
df2.loc[df2['PostalCode'] == 'M9V']

,PostalCode,Borough,Neighborhood,Latitude,Longitude
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437


- Toronto map

In [12]:
geolocator = Nominatim(user_agent="coursera")
address = 'Toronto'
try:
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    print('The geograpical coordinates of {} are {}, {}.'.format(address, latitude, longitude))
except AttributeError:
    print('Cannot find: {}, will drop index: {}'.format(address, index))

my_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df2['Latitude'], df2['Longitude'], df2['PostalCode']):
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(my_map)  
    
my_map

The geograpical coordinates of Toronto are 43.653963, -79.387207.


- Will analyze with 4Square

In [13]:
CLIENT_ID = '...' # Foursquare ID
CLIENT_SECRET = '...' # Foursquare Secret
VERSION = '20180605' # API version

- will focus on Etobicoke

In [15]:
df2.set_index('PostalCode', inplace = True) 
neighborhood_latitude = df2.loc['M9V']['Latitude']
neighborhood_longitude = df2.loc['M9V']['Longitude']

In [16]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=NE044IXTOCA202APNIBVZCC1NBTIV11D4JBNGW1P2ZRV3ONP&client_secret=F5LRZGFOSQPHT45EUX4DNN0MGYUDSROA4TQP1NMLR5RY4QLU&v=20180605&ll=43.739416399999996,-79.5884369&radius=500&limit=100'

In [17]:
results = requests.get(url).json()
# results

In [18]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [19]:
venues = results['response']['groups'][0]['items']

In [20]:
nearby_venues = json_normalize(venues) # flatten JSON
nearby_venues

,reasons.count,reasons.items,referralId,venue.categories,venue.id,venue.location.address,venue.location.cc,venue.location.city,venue.location.country,venue.location.crossStreet,venue.location.distance,venue.location.formattedAddress,venue.location.labeledLatLngs,venue.location.lat,venue.location.lng,venue.location.postalCode,venue.location.state,venue.name,venue.photos.count,venue.photos.groups
0,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4b04a05bf964a520c45522e3-0,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",4b04a05bf964a520c45522e3,1530 Albion Rd,CA,Etobicoke,Canada,at Finch Ave. W.,318,"[1530 Albion Rd (at Finch Ave. W.), Etobicoke ...","[{'label': 'display', 'lat': 43.74196806166214...",43.741968,-79.586639,M9V 1B4,ON,Sheriff's No Frills,0,[]
1,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4c633939e1621b8d48842553-1,"[{'id': '4bf58dd8d48988d1c5941735', 'name': 'S...",4c633939e1621b8d48842553,6210 Finch Ave. W Suite 103,CA,Etobicoke,Canada,at Albion Rd.,344,"[6210 Finch Ave. W Suite 103 (at Albion Rd.), ...","[{'label': 'display', 'lat': 43.74242142838322...",43.742421,-79.589471,M9V 3Y5,ON,Subway,0,[]
2,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4be70e26cf200f47e334153c-2,"[{'id': '4d4ae6fc7a7b7dea34424761', 'name': 'F...",4be70e26cf200f47e334153c,1530 Albion Rd.,CA,Etobicoke,Canada,at Kipling Ave. (Albion Centre),370,[1530 Albion Rd. (at Kipling Ave. (Albion Cent...,"[{'label': 'display', 'lat': 43.74120165509377...",43.741202,-79.584545,M9V 1B4,ON,Popeyes Louisiana Kitchen,0,[]
3,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4cd4738cdfb4a1cd4337535c-3,"[{'id': '5370f356bcbc57f1066c94c2', 'name': 'B...",4cd4738cdfb4a1cd4337535c,1530 Albion Rd,CA,Etobicoke,Canada,Near Finch Ave. W.,413,"[1530 Albion Rd (Near Finch Ave. W.), Etobicok...","[{'label': 'display', 'lat': 43.7416936, 'lng'...",43.741694,-79.584373,M9V 1B4,ON,The Beer Store,0,[]
4,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4be58dc4cf200f479154133c-4,"[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",4be58dc4cf200f479154133c,1530 Albion Rd,CA,Etobicoke,Canada,Albion Mall,438,"[1530 Albion Rd (Albion Mall), Etobicoke ON M9...","[{'label': 'display', 'lat': 43.74083239297218...",43.740832,-79.583347,M9V 1B4,ON,Shoppers Drug Mart,0,[]
5,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4d8ba6910c4e41bdaaf7667f-5,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",4d8ba6910c4e41bdaaf7667f,"1530 Albion Road, Unit T25",CA,Etobicoke,Canada,NaN,397,"[1530 Albion Road, Unit T25, Etobicoke ON M9V ...","[{'label': 'display', 'lat': 43.74156896801906...",43.741569,-79.584489,M9V 1B4,ON,Pizza Pizza,0,[]
6,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-5112b872e4b0c0a78d7bcd27-6,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",5112b872e4b0c0a78d7bcd27,1620 Albion road,CA,Toronto,Canada,Albion Road and Finch Ave,319,"[1620 Albion road (Albion Road and Finch Ave),...","[{'label': 'display', 'lat': 43.74184023292111...",43.741840,-79.590561,NaN,ON,Sunny Foodmart,0,[]
7,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4cd9d00734bb8cfa6576babf-7,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",4cd9d00734bb8cfa6576babf,6220 Finch Ave West,CA,Etobicoke,Canada,Albion,306,"[6220 Finch Ave West (Albion), Etobicoke ON M9...","[{'label': 'display', 'lat': 43.7420146612893,...",43.742015,-79.589690,M9V 0A1,ON,Tim Hortons,0,[]
8,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4c1951d6834e2d7f2d3a2a80-8,"[{'id': '4bf58dd8d48988d16e941735', 'name': 'F...",4c1951d6834e2d7f2d3a2a80,1530 Albion Road,CA,Toronto,Canada,NaN,404,"[1530 Albion Road, Toronto ON M9V 1B4, Canada]","[{'label': 'display', 'lat': 43.74163461048035...",43.741635,-79.584446,M9V 1B4,ON,McDonald's,0,[]
9,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4da73b855da37feee89a1143-9,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",4da73b855da37feee89a1143,1701 Martingrove Road,CA,Toronto,Canada,NaN,31

In [21]:
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Sheriff's No Frills,Grocery Store,43.741968,-79.586639
1,Subway,Sandwich Place,43.742421,-79.589471
2,Popeyes Louisiana Kitchen,Fried Chicken Joint,43.741202,-79.584545
3,The Beer Store,Beer Store,43.741694,-79.584373
4,Shoppers Drug Mart,Pharmacy,43.740832,-79.583347


In [22]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

10 venues were returned by Foursquare.


In [23]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [24]:
df2

,Borough,Neighborhood,Latitude,Longitude
PostalCode,,,,
M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
M1G,Scarborough,Woburn,43.770992,-79.216917
M1H,Scarborough,Cedarbrae,43.773136,-79.239476
M1J,Scarborough,Scarborough Village,43.744734,-79.239476
M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476


In [25]:
venues = getNearbyVenues(names=df2['Borough'],latitudes=df2['Latitude'],longitudes=df2['Longitude'])

Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
East York
East York
East Toronto
East York
East York
East York
East Toronto
East Toronto
East Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
North York
Central Toronto
Central Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
North York
North York
York
York
Downtown Toronto
Wes

In [26]:
print(venues.shape)
venues.head()

(2247, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Scarborough,43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,Scarborough,43.784535,-79.160497,RIGHT WAY TO GOLF,43.785177,-79.161108,Golf Course
2,Scarborough,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,Scarborough,43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
4,Scarborough,43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


In [27]:
venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Central Toronto,117,117,117,117,117,117
Downtown Toronto,1276,1276,1276,1276,1276,1276
East Toronto,128,128,128,128,128,128
East York,71,71,71,71,71,71
Etobicoke,69,69,69,69,69,69
Mississauga,10,10,10,10,10,10
North York,239,239,239,239,239,239
Queen's Park,44,44,44,44,44,44
Scarborough,91,91,91,91,91,91


In [28]:
print('There are {} unique categories.'.format(len(venues['Venue Category'].unique())))

There are 274 unique categories.


In [29]:
# one hot encoding
onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['Neighborhood'] = venues['Neighborhood'] 

onehot.head()

,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
onehot.shape

(2247, 274)

In [31]:
grouped = onehot.groupby('Neighborhood').mean().reset_index()
grouped

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Central Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.008547,0.000000,0.000000,0.008547,0.000000,0.000000,0.000000,0.000000,0.008547
1,Downtown Toronto,0.000000,0.000784,0.000784,0.000784,0.000784,0.000784,0.001567,0.001567,0.001567,...,0.000000,0.007837,0.001567,0.000000,0.005486,0.000000,0.006270,0.000784,0.001567,0.003918
2,East Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.023438
3,East York,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.014085,0.000000,0.014085,0.000000,0.000000,0.000000,0.014085
4,Etobicoke,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.014493,0.000000,0.000000
5,Mississauga,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,North York,0.004184,0.000000,0.000000,0.004184,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.004184,0.004184,0.008368,0.000000,0.000000,0.004184,0.012552,0.000000
7,Queen's Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.022727,0.000000,0.022727
8,Scarborough,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.010989,0.000000,0.000000,0.000000,0.000000,0.000000
9,West Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.010695,0.000000,0.000000,0.010695,0.000000,0.005348,0.000000,0.000000,0.005348


In [32]:
grouped.shape

(11, 274)

In [33]:
num_top_venues = 5

for hood in grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = grouped[grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Central Toronto----
            venue  freq
0     Coffee Shop  0.08
1  Sandwich Place  0.07
2            Park  0.05
3            Café  0.04
4     Pizza Place  0.04


----Downtown Toronto----
         venue  freq
0  Coffee Shop  0.09
1         Café  0.06
2   Restaurant  0.04
3       Bakery  0.03
4        Hotel  0.03


----East Toronto----
                venue  freq
0    Greek Restaurant  0.08
1         Coffee Shop  0.06
2                Café  0.04
3      Ice Cream Shop  0.04
4  Italian Restaurant  0.04


----East York----
                 venue  freq
0          Coffee Shop  0.06
1                 Park  0.06
2                 Bank  0.04
3  Sporting Goods Shop  0.04
4          Pizza Place  0.04


----Etobicoke----
                  venue  freq
0           Pizza Place  0.13
1        Sandwich Place  0.07
2              Pharmacy  0.06
3  Fast Food Restaurant  0.04
4           Coffee Shop  0.04


----Mississauga----
                      venue  freq
0               Coffee Shop   0.2
1   

In [34]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [35]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = grouped['Neighborhood']

for ind in np.arange(grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Coffee Shop,Sandwich Place,Park,Café,Pizza Place,Clothing Store,Sushi Restaurant,Dessert Shop,Restaurant,Pub
1,Downtown Toronto,Coffee Shop,Café,Restaurant,Hotel,Bakery,Italian Restaurant,Bar,Japanese Restaurant,Pizza Place,American Restaurant
2,East Toronto,Greek Restaurant,Coffee Shop,Café,Italian Restaurant,Ice Cream Shop,Park,Brewery,Yoga Studio,Sandwich Place,Light Rail Station
3,East York,Park,Coffee Shop,Burger Joint,Sporting Goods Shop,Bank,Grocery Store,Pizza Place,Sandwich Place,Beer Store,Fast Food Restaurant
4,Etobicoke,Pizza Place,Sandwich Place,Pharmacy,Café,Grocery Store,Gym,Fast Food Restaurant,Coffee Shop,Beer Store,Bakery
5,Mississauga,Coffee Shop,Hotel,Mediterranean Restaurant,Fried Chicken Joint,Sandwich Place,Burrito Place,American Restaurant,Gym / Fitness Center,Electronics Store,Eastern European Restaurant
6,North York,Coffee Shop,Fast Food Restaurant,Clothing Store,Pizza Place,Restaurant,Shopping Mall,Japanese Restaurant,Sandwich Place,Grocery Store,Park
7,Queen's Park,Coffee Shop,Sushi Restaurant,Gym,Diner,Café,Mexican Restaurant,Bar,Italian Restaurant,Japanese Restaurant,Spa
8,Scarborough,Fast Food Restaurant,Chinese Restaurant,Breakfast Spot,Bakery,Coffee Shop,Pizza Place,Playground,Thai Restaurant,Intersection,Sandwich Place
9,West Toronto,Bar,Café,Coffee Shop,Italian Restaurant,Pizza Place,Bakery,Restaurant,Breakfast Spot,Music Venue,Diner


In [36]:
# set number of clusters
kclusters = 5

grouped_clustering = grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 4, 4, 1, 0, 3, 4, 0], dtype=int32)

In [37]:
merged = grouped

# add clustering labels
merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
merged = merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

merged.head() # check the last columns!

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,Coffee Shop,Sandwich Place,Park,Café,Pizza Place,Clothing Store,Sushi Restaurant,Dessert Shop,Restaurant,Pub
1,Downtown Toronto,0.0,0.000784,0.000784,0.000784,0.000784,0.000784,0.001567,0.001567,0.001567,...,Coffee Shop,Café,Restaurant,Hotel,Bakery,Italian Restaurant,Bar,Japanese Restaurant,Pizza Place,American Restaurant
2,East Toronto,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,Greek Restaurant,Coffee Shop,Café,Italian Restaurant,Ice Cream Shop,Park,Brewery,Yoga Studio,Sandwich Place,Light Rail Station
3,East York,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,Park,Coffee Shop,Burger Joint,Sporting Goods Shop,Bank,Grocery Store,Pizza Place,Sandwich Place,Beer Store,Fast Food Restaurant
4,Etobicoke,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,Pizza Place,Sandwich Place,Pharmacy,Café,Grocery Store,Gym,Fast Food Restaurant,Coffee Shop,Beer Store,Bakery
